In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
# load google drive for files 
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Read the csv file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ Data_pol2/merged_griddata2019.csv')

data['date_time'] = data['CalendarDateKey'].astype(str) + data['interval_start']
print(data.tail()) #7 columns, including the Date. 

         Unnamed: 0   index  CalendarDateKey interval_start  n_validations  \
1356705     1356705  1372.0         20191128       08:30:00              7   
1356706     1356706  1372.0         20191129       06:00:00              6   
1356707     1356707  1372.0         20191129       06:30:00              7   
1356708     1356708  1372.0         20191129       07:30:00             13   
1356709     1356709  1372.0         20191129       08:00:00              5   

                date_time  
1356705  2019112808:30:00  
1356706  2019112906:00:00  
1356707  2019112906:30:00  
1356708  2019112907:30:00  
1356709  2019112908:00:00  


In [ ]:
import datetime
#process the matrix
df_for_training=data.pivot(index='date_time',
                  columns='index',
                  values='n_validations'
                  )
df_for_training.index = pd.to_datetime(df_for_training.index, format='%Y%m%d%H:%M:%S')

In [ ]:
df_for_training.tail()

index,0.0,1.0,2.0,3.0,4.0,10.0,12.0,13.0,19.0,20.0,...,1329.0,1334.0,1335.0,1342.0,1347.0,1348.0,1350.0,1361.0,1362.0,1372.0
date_time,,,,,,,,,,,,,,,,,,,,,
2019-11-30 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-30 20:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
2019-11-30 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN
2019-11-30 21:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-30 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN


In [ ]:
#% adding time and dates tages to the data
df_for_training['dates_weekday']=[d.weekday() for d in df_for_training.index]
df_for_training['time']=[(d.time().hour*60+d.time().minute)/30 for d in df_for_training.index]

In [ ]:
df_for_training.shape
df_for_training=df_for_training.fillna(0)
df_for_training = df_for_training.astype(float)
df_for_training=df_for_training.to_numpy()
X_train, y_train=df_for_training[0:4303,:],df_for_training[1:4304,:-2]
X_test, y_test=df_for_training[4304:-1,:],df_for_training[4305:,:-2]

In [ ]:

# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
  
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
  
# fit the regressor with x and y data
regressor.fit(X_train,y_train )  

RandomForestRegressor(random_state=0)

In [ ]:
# the function to measure the noises
def measure_the_error(actual,pred):
    rmse_list=[]
    mae_list=[]
    num=int(len(pred)/35)
    root_mean_square=np.sqrt(np.square(np.subtract(actual,pred)).mean(axis=1))
    mean_absolute_error=abs(np.subtract(actual,pred)).mean(axis=1)
    for i in range(num):
        cur=35*i
        rmse_list.append(root_mean_square[cur:cur+35].mean())
        mae_list.append(mean_absolute_error[cur:cur+35].mean())
    # mean_absolute_percentage=np.mean(np.abs((actual - pred)/actual))*100
    # rmse = math.sqrt(mean_square)
    return rmse_list,mae_list


## Accuracy

In [ ]:
#accuracy
#accuracy
Y_pred = regressor.predict(X_test)
rmse,mae = measure_the_error(Y_pred,y_test)


In [ ]:
mae

[9.8527906122449,
 8.648844489795918,
 5.540931020408164,
 8.210532653061223,
 8.353447346938774,
 8.561711020408163,
 8.180697142857143,
 9.084568571428571,
 6.858670612244897,
 5.339906122448979,
 9.128955918367344,
 8.832791836734694,
 8.280263265306122,
 9.397357959183672,
 8.82664530612245,
 6.313903673469388,
 4.656866122448979,
 8.04002530612245,
 9.220139591836734,
 8.056403265306121,
 8.590035510204082,
 9.007355510204082,
 6.285131836734695,
 4.571058775510204,
 8.739624081632654,
 8.795509387755102,
 8.92447836734694,
 9.908310612244897,
 12.760780408163267,
 9.407488571428573]

## Robustness

In [ ]:
import random
def AddNoise(real_demand,X_train,missing,noise_level):
    res=[]
    for i in range(len(real_demand)):

        noise_demand=np.array(list(real_demand[i,:-2])).reshape(-1,1)
        tag=real_demand[i,-2:]
        

        replace_Loc=np.where((X_train[:,-2]==tag[0])&(X_train[:,-1]==tag[1]))[0]
        replace_Loc=np.array([i for i in replace_Loc if i>30])
        #add missing 
        Loc=random.sample(range(0,700),int(700*missing)) # random locations to add noise
        replace=X_train[replace_Loc][:,Loc].mean(axis=0).reshape(-1,1)

        # noise_demand[Loc]=replace.reshape(-1,1)
        noise_demand[Loc]*=np.array(0)
        #add noise 
        val=np.array(random.choices(noise_level,k=700)).reshape(-1,1)
        noise_demand*=val

        res.append(noise_demand) 
    return np.array(res)



In [ ]:
def calculate(X_test,X_train,missing=0.6,magnitude=0.6):
    rmse_res=[]
    if missing!=0.6:
        for i in range(1,6):
            X_test_noise=AddNoise(X_test,X_train,missing,[1-i*0.1,1+i*0.1])
            X_test_noise=np.concatenate((X_test_noise.reshape(1050,-1),X_test[:,-2:]),axis=1)
            Y_pred_noise = regressor.predict(X_test_noise)
            rmse,mae = measure_the_error(Y_pred_noise,y_test)
            rmse_res.append(rmse)
    if magnitude!=0.6:
        for i in range(1,6):
            X_test_noise=AddNoise(X_test,X_train,i*0.1,[magnitude+1,1-magnitude])
            X_test_noise=np.concatenate((X_test_noise.reshape(1050,-1),X_test[:,-2:]),axis=1)
            Y_pred_noise = regressor.predict(X_test_noise)
            rmse,mae = measure_the_error(Y_pred_noise,y_test)
            rmse_res.append(rmse)
    return rmse_res



In [ ]:

rmse_res=calculate(X_test,X_train,missing=0.6,magnitude=0.3)

In [ ]:
rmse_res

[[67.57537773680613,
  52.74716928095968,
  38.62253686089491,
  80.57148637596788,
  78.95838169974161,
  79.74330760215285,
  88.95582043584724,
  88.79669001307181,
  50.34561158179414,
  32.26935262880181,
  83.062299062647,
  90.25211917559481,
  79.7485816018211,
  91.55296967204539,
  85.75361838769481,
  50.710290935457635,
  36.174280993421355,
  74.84444134134895,
  77.44390382330768,
  93.71440354324596,
  94.68111237987394,
  96.80205421689216,
  48.42795390224675,
  43.834892815794234,
  82.37941609482218,
  84.17738153140388,
  84.8966784692641,
  91.77190451622651,
  105.98656843887318,
  55.98796503174991],
 [85.95446454819286,
  65.09254980416212,
  42.30630145045128,
  98.57865541716156,
  102.40252036671829,
  95.0804795157343,
  119.07081897661189,
  105.21414992706411,
  64.16826588375531,
  36.22891520111281,
  92.632386092476,
  121.1722579205964,
  96.99359662497827,
  98.65293686929071,
  107.37707320083317,
  50.126211582762146,
  44.949879586068434,
  102.581